In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'kitchen_&_housewares'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 5023 palavras!


In [2]:
def normalizar(X):

    m, n = X.shape # m = qtde de objetos e n = qtde de atributos por objeto
    
    # Inicializa as variaves de saída
    X_norm = np.zeros( (m,n) ) #inicializa X_norm (base normalizada)
    #X_norm = lil_matrix((m, n))
    mu = 0 # inicializa a média
    sigma = 1 # inicializa o desvio padrão
      
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0, ddof=1)
    
    for i in range(m):
        for j in range(n):
            X_norm[i][j] = (X[i][j] - mu[j])/(sigma[j])
        
    return X_norm

Separa os dados em treinamento e teste:

In [3]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

#X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :]
Ytrain, Yval = Y2[train_index], Y2[test_index]

Seleciona features com chi-quadrado (a partir dos dados de treinamento):

In [4]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

Xtrain = Xtrain.toarray()
Xval = Xval.toarray()

In [5]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)

Número de features antes do chi-quadrado: 5023
----------------------------------------
Número de features após chi-quadrado: 436
['regret', 'larger', 'scale', 'compar', 'irregular', ('not', 'support'), ('not', 'need'), 'spare', 'disapoint', 'quiet', ('not', 'bother'), 'sharp', 'took', 'lucki', ('not', 'buy'), 'last', 'smooth', 'properli', 'con', 'take', 'warp', 'past', 'enjoy', 'highest', 'sharpen', 'ever', 'chines', 'feed', 'grip', 'horribl', 'not', 'decid', 'chicken', 'bark', 'fantast', 'wors', 'dvd', 'rust', 'call', 'caught', 'wok', 'sent', 'probe', 'correctli', ('not', 'effect'), 'aroma', ('not', 'give'), 'never', 'technic', 'brand', 'middl', ('not', 'think'), ('not', 'even'), 'concern', 'defin', 'favorit', 'pasta', 'wrote', 'shatter', 'leak', ('not', 'look'), 'exist', 'lodg', 'practic', 'long', ('not', 'make'), ('not', 'cook'), 'mattress', 'spout', 'defect', 'excit', 'overfil', 'fail', 'litter', 'set', 'nice', 'eleg', 'power', 'santoku', 'drip', ('not', 'work'), 'perfectli', 'din

In [6]:
def distancia(x, X):
               
    m = X.shape[0] 
    D = np.zeros(m) 

    i = 0
    for amostra in X:
        D = np.linalg.norm(amostra - x)
        i += 1
                    
    return D

In [7]:
def knn(x, X, Y, K):
        
    y = 0 
    
    ind_viz = np.ones(K, dtype=int)
    
    D = distancia(x, X)
    
    votos = np.zeros(len(set(Y)))

    ind_viz = np.argsort(D)[:K]
            
    for indice in ind_viz:
        votos[Y[indice]] += 1    
            
    y = np.argmax(votos)
        
    return y

In [14]:
def predicao(K, Xtrain, Ytrain, Xval, Yval):
    
    Ypred = []

    for i in range(Xval.shape[0]):
        y = knn(Xval[i], Xtrain, Ytrain, K)
        Ypred.append(y)
        
    acuracia = np.sum(Ypred==Yval)/len(Yval)
    
    return Ypred, acuracia


classes = np.unique(Y)

for k in range(3, 12, 2):
    print("K = " + str(k))
    Ypred, acuracia = predicao(k, Xtrain, Ytrain, Xval, Yval)
    print(acuracia)
    auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

K = 3
0.4875
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

0.4875
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [9]:
def curva_aprendizado(X, Y, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """
    
    k = 5

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        Ypred, acuracia = predicao(k, X[:i], Y[:i], X[:i], Y[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(k, X[:i], Y[:i], Xval, Yval)
        perf_val.append(acuracia)


    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()

In [10]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)

In [13]:
#K-folds
import k_folds as kf

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y2, nFolds, classes)

k = 1
resultados=[] # cria uma lista vazia para guardar os resultados obtidos em cada fold
for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (k) )
    
    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    totalFold = len(train_index)+len(test_index)
    
    X_train, X_test = X2[train_index, :], X2[test_index, :];
    Y_train, Y_test = Y2[train_index], Y2[test_index];
    
    X_train, new_vocabulary, index = pp.chi2(X_train, Y_train, vocabulary)
    X_test = X_test[:,index]

    X_train = X_train.toarray()
    X_test = X_test.toarray()
    
    for k in range(3, 12, 2):
        print("K = " + str(k))
        Ypred, acuracia = predicao(k, X_train, Y_train, X_test, Y_test)
        print(acuracia)
        auxResults = anl.relatorioDesempenho(Y_test, Ypred, classes, imprimeRelatorio=True)
    

    # adiciona os resultados do fold atual na lista de resultados
    resultados.append( auxResults ) 
        
    k+=1
    
kf.mediaFolds( resultados, classes )


-----------
1-fold: 
-----------

K = 3
0.5

	Revocacao   Precisao   F-medida   Classe
	1.000       0.500      0.667      0
	0.000       nan      nan      1
	------------------------------------------------
	0.500       nan      nan      Média macro
	0.500       0.500      0.500      Média micro

	Acuracia: 0.500
K = 5
0.5

	Revocacao   Precisao   F-medida   Classe
	1.000       0.500      0.667      0
	0.000       nan      nan      1
	------------------------------------------------
	0.500       nan      nan      Média macro
	0.500       0.500      0.500      Média micro

	Acuracia: 0.500
K = 7
0.5

	Revocacao   Precisao   F-medida   Classe
	1.000       0.500      0.667      0
	0.000       nan      nan      1
	------------------------------------------------
	0.500       nan      nan      Média macro
	0.500       0.500      0.500      Média micro

	Acuracia: 0.500
K = 9
0.5

	Revocacao   Precisao   F-medida   Classe
	1.000       0.500      0.667      0
	0.000       nan      nan      1